In [1]:
# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time
from datetime import datetime
from datetime import timedelta
import scipy as sp

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", context="talk")

# ibl specific things
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses import behavior as behavioral_analyses


Connecting anneurai@datajoint.internationalbrainlab.org:3306


In [2]:
# all mice that started training
# leave out mice in the brain-wide project that got too old before the ephys setup was ready...
all_mice = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') \
    * (subject.SubjectLab & 'lab_name = "churchlandlab"')
all_mice = all_mice.fetch(format='frame').reset_index()
print('# of animals in brainwide project:')
all_mice.subject_nickname.nunique()


# of animals in brainwide project:


15

In [3]:
# all mice that made it to ready4ephys
all_ephys_sess = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') \
    * (subject.SubjectLab & 'lab_name = "churchlandlab"') \
    * (acquisition.Session & 'task_protocol LIKE "%ephysChoice%"')
all_ephys_sess = all_ephys_sess.fetch(format='frame').reset_index()
print('# of animals with ephys sessions:')
all_ephys_sess.subject_nickname.nunique()


# of animals with ephys sessions:


10

In [4]:
animals_noephys = list(set(all_mice.subject_nickname.unique()) - set(all_ephys_sess.subject_nickname.unique()))
print('animals without any ephys data:')
sorted(animals_noephys)

animals without any ephys data:


['CSHL046', 'CSHL048', 'CSHL056', 'CSHL057', 'CSHL058']

#### animals that never made it to ephys 

* CSHL048: issue during headbar implant surgery, never started training
* CSHL056: experimental well from tip of centrifuge tube, infection on skull
* CSHL057: experimental well from tip of centrifuge tube, infection on skull

* CSHL046: has not reached ready4ephysRig, now > 7 months
* CSHL058: now on biasedCW, will hopefully still record


In [5]:
### for those mice with ephys data, how many sessions?
print('average number of sessions per mouse:')
print(all_ephys_sess.groupby(['subject_nickname'])['session_start_time'].nunique().mean())

all_ephys_sess.groupby(['subject_nickname'])['session_start_time'].nunique().reset_index().sort_values(by='subject_nickname')

average number of sessions per mouse:
4.9


,subject_nickname,session_start_time
0,CSHL045,6
1,CSHL047,6
2,CSHL049,8
3,CSHL051,4
4,CSHL052,6
5,CSHL053,6
6,CSHL054,2
7,CSHL055,2
8,CSHL059,5
9,CSHL060,4


### cull/ephys end reasons

##### bad animals
* CSHL054: session 1 great. session 2 terrible behavior, mouse looks like it's in pain
* CSHL055 (honeycomb): session 1 great, session 2 awful. mouse lost weight, lethargic, died 4 days after craniotomy surgery
* CSHL051: when attempting to do second cranio surgery with punch, skull broke. emergency perfusion.

##### good animals
* CSHL059: no more sites to go into the brain
* CSHL045: 
* CSHL047:
* CSHL052: honeycomb; quite a lot of blood in cranio but behavior great.
* CSHL053: 
* CSHL049: 
* CSHL060: still ingesting (has 6 sessions)


In [6]:
# how many probes per ephys session?
ephys = dj.create_virtual_module('ephys', 'ibl_ephys')

all_ephys_sess = (subject.Subject & 'subject_birth_date > "2019-03-01"') \
    * (subject.SubjectProject & 'subject_project = "ibl_neuropixel_brainwide_01"') \
    * (subject.SubjectLab & 'lab_name = "churchlandlab"') \
    * (acquisition.Session & 'task_protocol LIKE "%ephysChoice%"') \
    * ephys.ProbeInsertion * (ephys.ProbeTrajectory & 'insertion_data_source = "Micro-manipulator"')
all_ephys_sess = all_ephys_sess.fetch(format='frame').reset_index()

In [26]:
### for those mice with ephys data, how many sessions?
yield_permouse = all_ephys_sess.groupby(['subject_nickname'])['session_start_time', 'probe_trajectory_uuid'].nunique().reset_index()
yield_permouse.rename(columns={'session_start_time':'num_sessions', 'probe_trajectory_uuid':'num_penetrations'}, inplace=True)
assert ((yield_permouse['num_sessions'] * 2 >= yield_permouse['num_penetrations']).all() == True)

print('average number of penetrations per mouse:')
print(yield_permouse['num_penetrations'].mean())

yield_permouse.sort_values(by='subject_nickname')

average number of penetrations per mouse:
7.0


,subject_nickname,num_sessions,num_penetrations
0,CSHL045,4,7
1,CSHL047,5,9
2,CSHL049,6,9
3,CSHL051,4,8
4,CSHL052,5,8
5,CSHL053,4,7
6,CSHL054,2,3
7,CSHL055,2,4
8,CSHL059,5,9
9,CSHL060,3,6
